In [1]:
import pandas as pd
import numpy as np
import nltk 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from scipy.sparse import csc_matrix
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.lancaster import LancasterStemmer



In [2]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()

db = client.strain_project

db.strains

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'strain_project'), 'strains')

In [3]:
strain_initial = pd.DataFrame(db.strains.find())

In [4]:
strain_initial.head()

,_id,username,url,review,strain_name
0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle
1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle
2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf
3,5dc33349e1127a705e2ce27c,mtn420man,https://www.leafly.com/hybrid/longbottom-leaf/...,"“Such a smooth smoke. Instant head high, very ...",longbottom-leaf
4,5dc33349e1127a705e2ce27d,rjjia,https://www.leafly.com/hybrid/longbottom-leaf/...,“I now truly understand the beauty of middle e...,longbottom-leaf


In [5]:
strain_initial.shape

(898280, 5)

In [6]:
strain_initial=strain_initial.drop_duplicates(subset="review")

In [7]:
strain_initial.reset_index()

,index,_id,username,url,review,strain_name
0,0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle
1,1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle
2,2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf
3,3,5dc33349e1127a705e2ce27c,mtn420man,https://www.leafly.com/hybrid/longbottom-leaf/...,"“Such a smooth smoke. Instant head high, very ...",longbottom-leaf
4,4,5dc33349e1127a705e2ce27d,rjjia,https://www.leafly.com/hybrid/longbottom-leaf/...,“I now truly understand the beauty of middle e...,longbottom-leaf
...,...,...,...,...,...,...
165926,224565,5dc3335ee1127a705e304fae,MizzGizzi,https://www.leafly.com/hybrid/blue-magoo/revie...,“I really liked this strain for my day time. I...,blue-magoo
165927,224566,5dc3335ee1127a705e304faf,RecWreck,https://www.leafly.com/hybrid/blue-magoo/revie...,“Made me giggly with a very noticeable headkic...,blue-magoo
165928,224567,5dc3335ee1127a705e304fb0,samboogie,https://www.leafly.com/hybrid/blue-magoo/revie...,“just hit the bammer with this bud wow i feel ...,blue-magoo
165929,224568,5dc3335ee1127a705e304fb1,dapperErl,https://www.leafly.com/hybrid/blue-magoo/revie...,“The Blue Magoo from Forbidden Farms is FIRE!!...,blue-magoo


In [8]:
strain_initial.shape

(165931, 5)

In [9]:
strain_1 = strain_initial.copy()

In [10]:
strain_1.isna().any()

_id            False
username       False
url             True
review          True
strain_name    False
dtype: bool

In [11]:
strain_1=strain_1.dropna(subset=['review','url'])

In [12]:
strain_1.isna().any()

_id            False
username       False
url            False
review         False
strain_name    False
dtype: bool

In [13]:
strain_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165928 entries, 0 to 224569
Data columns (total 5 columns):
_id            165928 non-null object
username       165928 non-null object
url            165928 non-null object
review         165928 non-null object
strain_name    165928 non-null object
dtypes: object(5)
memory usage: 7.6+ MB


In [14]:
strain_1['strain_name'].value_counts()

sour-diesel          3166
gsc                  3108
green-crack          2764
og-kush              2354
original-glue        2081
                     ... 
cbd-rich-blessing       1
agape                   1
orange-snow-cone        1
oldtimers-haze          1
tangenesia              1
Name: strain_name, Length: 2543, dtype: int64

In [15]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [16]:
strain_count=pd.DataFrame(strain_1['strain_name'].value_counts())

In [17]:
strain_count.head(50)

,strain_name
sour-diesel,3166
gsc,3108
green-crack,2764
og-kush,2354
original-glue,2081
granddaddy-purple,2045
jack-herer,1836
white-widow,1710
bubba-kush,1634
pineapple-express,1520


In [18]:
import re # Regular expression library
import string

In [19]:
strain_1['clean_review']=strain_1['review']
strain_1.head(3)

,_id,username,url,review,strain_name,clean_review
0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle,“40% head 60% body. Nice flavor 20 or 30 min b...
1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle,"“Love this strain. Happy, Calm, Relaxed but no..."
2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf,“Are you looking to get shire-smacked in the f...


In [39]:
def clean_text (text):
    new_text= re.sub('[%s]' % re.escape(string.punctuation), ' ',text) 
    new_text=re.sub('\w*\d\w*', ' ',new_text)
    new_text=new_text.lower()
    new_text=re.sub(r"[^a-zA-Z0-9]+", ' ',new_text)
    stemmer=LancasterStemmer()
    new_text=stemmer.stem(new_text)
    
    return new_text

In [40]:
strain_1['clean_review'] = strain_1['review'].apply(lambda x: clean_text(x))

In [33]:
strain_1.head(5)

,_id,username,url,review,strain_name,clean_review
0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle,head body nice flavor or min before full affe...
1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle,love this strain happy calm relaxed but not g...
2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf,are you looking to get shire smacked in the f...
3,5dc33349e1127a705e2ce27c,mtn420man,https://www.leafly.com/hybrid/longbottom-leaf/...,"“Such a smooth smoke. Instant head high, very ...",longbottom-leaf,such a smooth smoke instant head high very me...
4,5dc33349e1127a705e2ce27d,rjjia,https://www.leafly.com/hybrid/longbottom-leaf/...,“I now truly understand the beauty of middle e...,longbottom-leaf,i now truly understand the beauty of middle e...


In [34]:
strain_1.loc[strain_1["review"].str.contains("fire")]

,_id,username,url,review,strain_name,clean_review
169,5dc33349e1127a705e2ce322,Chill_Panda,https://www.leafly.com/indica/flaming-cookies/...,“This is a really great strain fire go which i...,flaming-cookies,this is a really great strain fire go which i...
178,5dc33349e1127a705e2ce32b,jakeprnhs,https://www.leafly.com/indica/flaming-cookies/...,“So what I smoked was called fire scouts. I be...,flaming-cookies,so what i smoked was called fire scouts i bel...
453,5dc33349e1127a705e2ce43e,MyJokerHandStayLIT,https://www.leafly.com/hybrid/slimer-og/review...,“Slymer or (Slimer) from Moxie! I’ve literally...,slimer-og,slymer or slimer from moxie i ve literally be...
683,5dc33349e1127a705e2ce524,OldLion,https://www.leafly.com/indica/afghani/reviews/...,“I was in the US Army for 3 years until I deci...,afghani,i was in the us army for years until i decide...
988,5dc33349e1127a705e2ce655,ducky518,https://www.leafly.com/indica/afghani/reviews/...,"“My favorite strain, this shit is fire af🔥👌”",afghani,my favorite strain this shit is fire af
...,...,...,...,...,...,...
224187,5dc3335ee1127a705e304e34,moemiami,https://www.leafly.com/hybrid/irene-kush/revie...,“Great for daytime use. My synapses fire so b...,irene-kush,great for daytime use my synapses fire so bea...
224334,5dc3335ee1127a705e304ec7,Statusdropout,https://www.leafly.com/hybrid/blue-magoo/revie...,“I enjoyed the white magoo. It was some good f...,blue-magoo,i enjoyed the white magoo it was some good fire
224455,5dc3335ee1127a705e304f40,McCaul,https://www.leafly.com/hybrid/blue-magoo/revie...,“this shit is fire as fuck can't keep it!”,blue-magoo,this shit is fire as fuck can t keep it
224556,5dc3335ee1127a705e304fa5,tiffpankey,https://www.leafly.com/hybrid/blue-magoo/revie...,"“Acquired a premium harvest of this strain, 25...",blue-magoo,acquired a premium harvest of this strain thc...


In [24]:
strain_1.loc[strain_1["username"]=='ducky518']

,_id,username,url,review,strain_name,clean_review
988,5dc33349e1127a705e2ce655,ducky518,https://www.leafly.com/indica/afghani/reviews/...,"“My favorite strain, this shit is fire af🔥👌”",afghani,<LancasterStemmer>


In [25]:
strain_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165928 entries, 0 to 224569
Data columns (total 6 columns):
_id             165928 non-null object
username        165928 non-null object
url             165928 non-null object
review          165928 non-null object
strain_name     165928 non-null object
clean_review    165928 non-null object
dtypes: object(6)
memory usage: 8.9+ MB


## LSA

In [26]:
stop_words = text.ENGLISH_STOP_WORDS.union(["little","great","love","happy","effects","time","feel","feeling","definitely","strains","pretty","good","indica","sativa","high","strain","like","smoke","buds","bud","just","smoking","smoke","really","pretty","makes","ve","got"])

In [27]:
vectorizer = CountVectorizer(stop_words=stop_words,max_df=.7,min_df=.045,ngram_range=(1, 2))
doc_word = vectorizer.fit_transform(strain_1['clean_review'])
doc_word

AttributeError: 'LancasterStemmer' object has no attribute 'lower'

In [ ]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(4),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lsa, vectorizer.get_feature_names(), 10)

In [ ]:
Vt = pd.DataFrame(doc_topic.round(4),
             index = strain_1['clean_review'],
             columns = ["component_1","component_2","component_3","component_4"])
Vt

## NMF

In [ ]:
nmf_model = NMF(3)
doc_topic = nmf_model.fit_transform(doc_word)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2","component_3"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, vectorizer.get_feature_names(), 5)

In [ ]:
H = pd.DataFrame(doc_topic.round(5),
             index = strain_1['clean_review'],
             columns = ["component_1","component_2","component_3" ])
H

In [ ]:
doc_word2=doc_word.copy()
doc_word2 = pd.DataFrame(doc_word2.toarray())

In [ ]:
doc_word2['strain_name'] = strain_1["strain_name"]


In [ ]:
doc_word2.shape

In [ ]:
doc_word2.head()

In [ ]:
strain_groupby=doc_word2.groupby(['strain_name']).sum()
strain_groupby

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(strain_groupby)


In [ ]:
doc_word_2 = tfidf_transformer.fit_transform(strain_groupby)
doc_word_2

In [ ]:
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(doc_word_2)
lsa.explained_variance_ratio_

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(4),
             index = ["component_1","component_2","component_3","component_4"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lsa, vectorizer.get_feature_names(), 10)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sscale = StandardScaler()

In [ ]:
pd.DataFrame(doc_topic)

In [ ]:
strain_cosine=pd.DataFrame(cosine_similarity(doc_topic),columns=strain_groupby.index.values,index=strain_groupby.index.values)

In [ ]:
strain_cosine['gelato'].sort_values(ascending=False)

In [ ]:
lsa.explained_variance_ratio_

## LDA

In [ ]:
strain_sparse=csc_matrix(strain_groupby)

In [ ]:
strain_sparse.shape

In [ ]:
strain_sparse

In [ ]:
corpus = matutils.Sparse2Corpus(strain_sparse)

In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=5)

In [ ]:
len(id2word)

In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=3,passes=5)

## LDA with Sklearn

In [ ]:
lda_model = LatentDirichletAllocation(n_components=4,max_iter=100)
lda_doc_topics=lda_model.fit_transform(strain_groupby)

In [ ]:
lda_model.components_[0]

In [ ]:
len(lda_model.components_)

In [ ]:
topic_word_lda = pd.DataFrame(lda_model.components_.round(4),
             index = ["component_1","component_2","component_3","component_4"],
             columns = vectorizer.get_feature_names())
topic_word_lda

In [ ]:
display_topics(lda_model, vectorizer.get_feature_names(), 10)